# Track data lineage: `Pipeline`, `Notebook`, `Run`


## What is a `Run`?

{class}`~lamindb.File` are transformed by instances of {class}`~lamindb.Run`. They are the {attr}`~lamindb.Run.inputs` and {attr}`~lamindb.Run.outputs` of runs.

Conversely, the {attr}`~lamindb.File.source` of {class}`~lamindb.File` is always the output of a run!

A `Run` itself points to the {class}`~lamindb.Transform` object that's being run, either a `notebook` or a `pipeline`.

In [ ]:
import lamindb as ln

## Notebook run

The metadata of Jupyter notebooks is automatically detected and `ln.Run` assumes `global_context=True`: we don't need to keep track of the run record ourselves, but can access it via `ln.context`:

In [ ]:
ln.track()

In [ ]:
ln.context.run

Let us query where `File` "iris_new" has been ingested:

In [ ]:
ln.select(ln.Transform).join(ln.Run).join(ln.File, name="iris_new").first()

Alternatively, you can query for the run that contains a notebook attribute:

In [ ]:
with ln.Session() as ss:
    file = ss.select(ln.File, name="iris_new").one()
    print(file.source.transform)

## Pipeline run

In [ ]:
filepath = ln.dev.datasets.file_fastq()

When working with a pipeline, we'll register it before running it.

In [ ]:
transform = ln.add(ln.Transform(name="10x scRNA-seq nextseq", v="1"))

transform

We can then use the {func}`~lamindb.track` as before (if we don't register a pipeline with the correct name, we'll be asked to):

In [ ]:
ln.track(transform=transform)

In [ ]:
file_fastq = ln.File(filepath)

In [ ]:
ln.add(file_fastq)

We can also manually pass a run and not use the global run context (`ln.context`) set by `ln.track`:
```
run = ln.Run(transform=transform, name="ingest-fastq")
ln.File(filepath, source=run)
```

## Track run inputs

While run outputs are automatically tracked as data sources, run inputs aren't.

However, you can simply call `is_run_input` upon loading `File`.

Let's register a downstream pipeline:

In [ ]:
ln.track(transform=ln.Transform(name="Cell Ranger", v="7"))

Let's query input data for this pipeline, a fastq.

To process in the pipeline, we need to `load()` it (download it from the cloud and access the on-disk or in-memory representation).

To track it as an input for the current run, set `is_run_input=True`.

In [ ]:
with ln.Session() as ss:
    file_fastq = ss.select(ln.File, name="input").one()
    file_fastq.load(is_run_input=True)

In [ ]:
file_fastq.targets

In [ ]:
ln.add(file_fastq)

In [ ]:
output_filepath = ln.dev.datasets.file_bam()

In [ ]:
output_filepath

In [ ]:
file = ln.File(output_filepath)

ln.add(file)

## Data lineage

Now let's track from which files that the `output.bam` file is generated, aka, the input file of the run that produced file `output.bam`

In [ ]:
with ln.Session() as ss:
    run = ss.select(ln.Run).join(ln.File, name="output", suffix=".bam").one()
    assert run.inputs[0].name == "input"
    print(run.inputs)